In [4]:
search_for = 124
start_from = 0
threads = 10
thread = 8
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.004059553146362305
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 75
all cases: 621252555
type: [1, 1, 1, 1, 124] 124
cases of this type: 236421376
100000 26.488312347264714
200000 50.09829530858981
300000 56.4314587642328
400000 47.60390570968578
500000 58.99485032915619
600000 57.855806589241595
700000 55.627751443818426
800000 49.311989333479964
900000 56.425317282507066
1000000 19.06570305088083
1100000 55.24465907535822
1200000 43.95766885948971
1300000 47.51426987978123
1400000 44.6709357182611
1500000 49.47261208223641
1600000 45.69167835767783
1700000 45.045164682266105
1800000 48.97820660420455
1900000 38.47267526508298
2000000 23.460708048015825
2100000 44.8436404866747
2200000 42.276053314149465
2300000 45.504291699019696
2400000 43.243554649948955
2500000 46.50990010657234
2600000 41.32445726894564
2700000 42.95723468158997
2800000 43.88235653255906
2900000 17.23650963794498
3000000 41.853750171

29800000 40.87035304051681
29900000 40.67719543809513
30000000 42.80392266094233
30100000 44.25112057778421
30200000 44.37094768831703
30300000 27.502052911683293
30400000 38.96273993518707
30500000 46.104396587703356
30600000 45.610542448590714
30700000 45.061228798809154
30800000 19.98196477867042
30900000 46.486899991520794
31000000 46.364879794730264
31100000 43.90313706836286
31200000 45.505072731071024
31300000 41.40640400991922
31400000 40.98756703714534
31500000 17.209983863234044
31600000 41.10402727434521
31700000 40.751572095359826
31800000 42.72625680192004
31900000 43.54411080771203
32000000 45.8031540907461
32100000 45.59938561878482
32200000 24.48690565258869
32300000 43.040660881073556
32400000 45.39077978833035
32500000 45.41817240950726
32600000 44.16275281199311
32700000 26.273283848754154
32800000 38.547269321376895
32900000 45.308763593722524
33000000 42.284265395173726
33100000 41.36620374323989
33200000 39.86643906703344
33300000 38.45953499414935
33400000 18.259

59800000 37.17441492543865
59900000 36.549423935392696
60000000 37.16664565021035
60100000 15.131917678682832
60200000 38.36408752912103
60300000 36.10914921178967
60400000 37.6298830274915
60500000 36.63574740537928
60600000 20.490260943151416
60700000 39.18450323247353
60800000 39.95049452926293
60900000 40.47515828279689
61000000 43.43069213475134
61100000 43.002431788726895
61200000 42.43435457074648
61300000 42.45328030742256
61400000 40.75909384216281
61500000 32.59784521810513
61600000 25.070963042947696
61700000 36.713924530994625
61800000 39.342918351070225
61900000 36.973069420052056
62000000 17.12936969360823
62100000 35.21995618985522
62200000 38.979290095699106
62300000 36.58779328136417
62400000 37.61351752591803
62500000 21.55720111295755
62600000 40.21731781987348
62700000 40.791454275218726
62800000 41.66444799787346
62900000 42.21929414582982
63000000 41.6054771556359
63100000 41.66540338198709
63200000 41.34342766277243
63300000 39.00042544616423
63400000 39.68997544

89800000 38.25545510145079
89900000 36.31142386858688
90000000 34.99853900791473
90100000 35.15725797285153
90200000 35.45404714296453
90300000 34.05264621696972
90400000 16.82498517949789
90500000 32.62305120518729
90600000 15.252646283040017
90700000 32.024242803824826
90800000 26.6466141204898
90900000 26.3895707158719
91000000 35.034474079696665
91100000 34.68981596694062
91200000 35.50941560444762
91300000 35.055094689091064
91400000 37.360019798373266
91500000 37.865779754609854
91600000 39.89705260838369
91700000 39.996425435650664
91800000 38.48621310757299
91900000 36.468479863961406
92000000 36.95068786356422
92100000 36.455882688034045
92200000 36.301175174709286
92300000 16.608122203023935
92400000 36.9599722692731
92500000 17.800789571038365
92600000 31.926452119975085
92700000 25.27962741693019
92800000 29.333019110821304
92900000 35.76748218035672
93000000 36.53552022480008
93100000 36.29855920402248
93200000 36.363417984834555
93300000 39.122090090593154
93400000 40.373

119200000 3.822458901146074
119300000 3.768629732753535
119400000 3.7634365640007257
119500000 3.8036250532728504
119600000 3.9618307205755343
119700000 3.814826856047014
119800000 3.840712208042923
119900000 3.8559107538279758
120000000 3.865026285191516
120100000 3.8184898217875527
120200000 3.791303939156509
120300000 3.777373768487095
120400000 3.830388218237774
120500000 3.831711420547303
120600000 3.814370759805044
120700000 3.821738656082961
120800000 3.8170946920347744
120900000 3.81288703673621
121000000 3.804822721308036
121100000 3.7920262788400816
121200000 3.743017378644168
121300000 3.7686139246740042
121400000 3.7738019541551076
121500000 3.8311866880896193
121600000 3.8299776489538218
121700000 3.82597219145092
121800000 3.8319117693979545
121900000 3.790630155486471
122000000 3.873345562394672
122100000 3.8444327849451003
122200000 3.8314829893517426
122300000 3.8516950779876047
122400000 3.8146146903194587
122500000 3.8994652416371105
122600000 3.8471824227851497
1227

...

493100000 4.0426110419479375
493200000 3.892256443061402
493300000 3.6540082982649706
493400000 1.4431942828155426
493500000 3.4714776986304123
493600000 3.0438343916928834
493700000 2.8702924004887644
493800000 3.4674189689567685
493900000 3.7115309411137236
494000000 3.737618731649157
494100000 4.147090098246117
494200000 3.543489374000344
494300000 2.4118329143679142
494400000 2.5173498353024026
494500000 3.2245909799719352
494600000 2.481092137404233
494700000 3.467150391772807
494800000 3.664047195302854
494900000 3.7451804893839564
495000000 3.5973709576506416
495100000 3.8583445123490625
495200000 3.7194156996727457
495300000 1.2858705851140124
495400000 3.3804387213797074
495500000 2.26010442541184
495600000 3.4075777252128128
495700000 3.508780003334744
495800000 3.6013060466607443
495900000 3.526763626378974
496000000 3.5522789138696917
496100000 3.8699053420407874
496200000 2.3919656109716296
496300000 2.41934741921453
496400000 2.5566745075044564
496500000 2.998834581978321

521000000 3.196279708411581
521100000 2.968097864982555
521200000 2.9795264356279407
521300000 2.907420230251398
521400000 2.932987025711854
521500000 3.098021789031264
521600000 3.1035845609801247
521700000 2.9136957926561537
521800000 2.9857963472804228
521900000 2.990580852954871
522000000 3.2107727359701297
522100000 3.1579288517861928
522200000 3.0887706227245233
522300000 3.064050765984237
522400000 3.2289039801266655
522500000 3.2068541658933722
522600000 3.173033678833276
522700000 3.0733056694454914
522800000 3.241473283616811
522900000 3.1417170935470566
523000000 3.1057859861316315
523100000 3.0214834289899604
523200000 3.200616913985858
523300000 3.1150580209496717
523400000 3.1628956689249543
523500000 2.800745846175581
523600000 3.027590684179604
523700000 3.0331082402270466
523800000 2.8746032168003226
523900000 2.9182640818962287
524000000 2.9294082252333267
524100000 2.757920488860826
524200000 3.506831328769197
524300000 2.8443546553208203
524400000 3.679922835264057


547800000 3.5014087324799004
547900000 2.976451816246268
548000000 1.707852213439445
548100000 2.654346541648279
548200000 3.4215915508208075
548300000 1.0084280905788574
548400000 0.5394394150814613
548500000 0.5390871647012333
548600000 0.5397441363896165
548700000 0.5382138832533061
548800000 0.5343260713435114
548900000 0.5362198646566935
549000000 0.5384922069039146
549100000 0.5475861296397546
549200000 0.5400956449724402
549300000 0.5372455937999685
549400000 0.5304787188520498
549500000 0.5305970235791472
549600000 0.5295538862013519
549700000 0.5287701419725617
549800000 0.5281645921636223
549900000 0.5391399070619146
550000000 0.6493063544221189
550100000 0.6957278627164397
550200000 0.655229216779977
550300000 0.5250257946765224
550400000 0.5263386024243004
550500000 0.5229671688895225
550600000 0.5390124467360047
550700000 3.308337599815677
550800000 3.162380202940881
550900000 1.375961087898665
551000000 3.5733851244275434
551100000 3.2596046111510595
551200000 2.692912748

576100000 0.9274449178052743
576200000 0.9123499191813734
576300000 1.2950008118721148
576400000 1.2182332734847134
576500000 1.0287716535242564
576600000 0.9399763024786413
576700000 1.146150730088214
576800000 1.2951218651629288
576900000 1.0099248388739528
577000000 1.1825682988444863
577100000 0.5779087506880032
577200000 1.2994905792194904
577300000 1.5433077902431422
577400000 0.9529323904445272
577500000 1.1446538440918923
577600000 0.6834393350138764
577700000 1.3083246023519965
577800000 1.1372089669270615
577900000 1.2667000531582269
578000000 0.9871102364572915
578100000 0.8645612956621648
578200000 1.3100230578576195
578300000 0.9295153110453819
578400000 1.2259864401815235
578500000 0.7445367788502442
578600000 1.3346847432176372
578700000 1.3128800363872946
best so far: 0
type: [1, 2, 1, 31, 2] 124
cases of this type: 953312
578800000 0.4154865248516036
578900000 0.27484839532961447
579000000 0.27384753060418365
579100000 0.27395123014176886
579200000 0.273642408636517
57

599500000 0.1608717684189909
599600000 0.16068070260171097
599700000 0.16058542843797802
599800000 0.4116759404898319
599900000 0.7768558040372067
600000000 0.9801896222848894
600100000 0.8774740640376011
600200000 0.5758996421474848
600300000 0.8159581379254064
600400000 0.631464307803058
600500000 0.8046478909592827
600600000 0.7246123434516191
600700000 0.6075446402534578
600800000 0.8066118326318562
600900000 0.6496693377853333
601000000 0.7995118165774843
601100000 0.607334036255588
601200000 0.7067322475785613
601300000 0.7285039227716757
601400000 0.6547756485726172
601500000 0.7874469804997046
601600000 0.5559479434270495
601700000 0.7453513907187184
601800000 0.6771170423683226
601900000 0.6924274717252751
602000000 0.7256289586874412
602100000 0.5680607300767998
602200000 0.7975232263927262
602300000 0.661021934408466
602400000 0.6850392380225261
602500000 0.7014374593201578
602600000 0.5615157011349069
602700000 0.7542065796814859
602800000 0.6843258951698078
602900000 0.692